# Outras Camadas Convolucionais

Além das camadas convolucionais padrões, algumas outras surgiram tentando melhorar alguns quesitos da convolução tradicional.

Entre elas estão:

- [convolução dilatada](https://arxiv.org/abs/1511.07122) (*dilated convolution*), que permite que o filtro convolucional tenha *buracos* aumentando o *receptive field* mas mantendo a resolução da imagem, e 
- [convolução separável por canal](https://arxiv.org/abs/1704.04861) (*depthwise separable convolution*), que, de acordo com a literatura, alcança resultados similares ao da convolução padrão porém usando menos parâmetros (por isso, são mais rápidas).

Antes de começar, vamos instalar o MXNet. Esse pequeno bloco de código abaixo é usado somente para instalar o MXNet para CUDA 10. Execute esse bloco somente uma vez e ignore possíveis erros levantados durante a instalação.

**ATENÇÃO: a alteração deste bloco pode implicar em problemas na execução dos blocos restantes!**

In [1]:
!pip install mxnet-cu100

# imports basicos
import time, os, sys, numpy as np
import mxnet as mx
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, utils as gutils, data as gdata

# Tenta encontrar GPU
def try_gpu():
    try:
        ctx = mx.gpu()
        _ = nd.zeros((1,), ctx=ctx)
    except mx.base.MXNetError:
        ctx = mx.cpu()
    return ctx

ctx = try_gpu()
ctx

gpu(0)

In [0]:
## carregando dados

# código para carregar o dataset do MNIST
# http://yann.lecun.com/exdb/mnist/
def load_data_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'mnist')):
    """Download the MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.MNIST(root=root, train=True)
    mnist_test = gdata.vision.MNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# código para carregar o dataset do Fashion-MNIST
# https://github.com/zalandoresearch/fashion-mnist
def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(
        '~', '.mxnet', 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [gdata.vision.transforms.Resize(resize)]
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    return train_iter, test_iter

# funções básicas
def _get_batch(batch, ctx):
    """Return features and labels on ctx."""
    features, labels = batch
    if labels.dtype != features.dtype:
        labels = labels.astype(features.dtype)
    return (gutils.split_and_load(features, ctx),
            gutils.split_and_load(labels, ctx), features.shape[0])

# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss, ctx=[mx.cpu()]):
    """Evaluate accuracy of a model on the given data set."""
    if isinstance(ctx, mx.Context):
        ctx = [ctx]
    acc_sum, n, l = nd.array([0]), 0, 0
    for batch in data_iter:
        features, labels, _ = _get_batch(batch, ctx)
        for X, y in zip(features, labels):
            # X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            y = y.astype('float32')
            y_hat = net(X)
            l += loss(y_hat, y).sum()
            acc_sum += (y_hat.argmax(axis=1) == y).sum().copyto(mx.cpu())
            n += y.size
        acc_sum.wait_to_read()
    return acc_sum.asscalar() / n, l.asscalar() / n
  
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, ctx,
                   num_epochs):
    print('training on', ctx)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.as_in_context(ctx), y.as_in_context(ctx)
            with autograd.record():
                y_hat = net(X)
                l = loss(y_hat, y).sum()
            l.backward()
            trainer.step(batch_size)
            y = y.astype('float32')
            train_l_sum += l.asscalar()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().asscalar()
            n += y.size
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss, ctx)
        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_loss, 
                 test_acc, time.time() - start))

## Convolucional Dilatada (*Dilated Convolution*)

Nas camadas convolucionais dilatadas, os pesos do filtro são empregados de maneira diferente quando comparados às convoluções padrão.
Especificamente, os filtros dessa camada não precisam ser contíguos e podem ter lacunas (ou "buracos") entre seus parâmetros.
Essas lacunas, inseridas de acordo com a taxa de dilatação $r \in \mathbb{N}$, que permite um aumento do tamanho do filtro convolucional, preservando o número de parâmetros treináveis, uma vez que os buracos inseridos não são considerados no processo de convolução.
Portanto, esta taxa de dilatação $ r $ pode ser vista como um parâmetro responsável por definir o alinhamento final dos pesos.

Formalmente, uma convolução dilatada 2-D recebe uma entrada em duas dimensões $ Y $, uma taxa de dilatação $ R $ e um vetor de peso 2-D $ W $ (neste caso, com tamanho $ n \times n $) e processos eles dessa forma: 

$$ Y[k, l] = \sum_{i=1}^{n} \sum_{j=1}^{n} X(k + r \times i, l + r \times j) K(i,j) $$
, onde $Y$ é a saída ou *feature map*.
Observe as diferenças entre a dilatação e a convolução padrão apresentadas na aula anterior.

O efeito da taxa de dilatação diferente $ r $ é apresentado na figura abaixo.
Como pode ser visto, taxas menores resultam em um filtro mais clusterizado (na verdade, a taxa 1 gera um filtro idêntico à convolução padrão) enquanto as taxas maiores fazem uma expansão do filtro, produzindo um kernel maior com vários buracos.
Como todo esse processo de dilatação do filtro é independente dos dados de entrada, alterar a taxa de dilatação não afeta a resolução do resultado, ou seja, em uma convolução dilatada, independente da taxa, a entrada e a saída têm a mesma resolução (considerando, claro, o efeito do *padding* e do *stride*).

<p align="center">
  <img width="500" src="https://drive.google.com/uc?export=view&id=1HixSXbx3HKWUrcCqJ8d-IF-RTykoT2Pz">
</p>

<p align="center">
  <img src="https://cdn-images-1.medium.com/max/800/0*oX5IPr7TlVM2NpEU.gif">
  <img src="https://cdn-images-1.medium.com/max/800/0*3cTXIemm0k3Sbask.gif">
</p>

Ao ampliar o filtro (com essas lacunas), a rede expande seu campo receptivo (já que os pesos serão organizados em uma forma mais esparsa), mas preserva a resolução e nenhuma redução de resolução nos dados é executada.
Portanto, esse processo tem várias vantagens, como:
(i) suporta a expansão do campo receptivo sem aumentar o número de parâmetros treináveis por camada, o que reduz a carga computacional, e
(ii) preserva a resolução do mapa de características, o que pode ajudar a rede a extrair informações ainda mais úteis dos dados, principalmente de pequenos objetos.

### Campo Receptivo (*Receptive Field*)

O campo receptivo é definido como a região no espaço de entrada que tem influência sobre a saída atual da rede convolucional.

<p align="center">
  <img src="https://miro.medium.com/max/1000/1*mModSYik9cD9XJNemdTraw.png">
</p>

### Implementação

No MXNet, a [camada convolução padrão](https://beta.mxnet.io/api/gluon/_autogen/mxnet.gluon.nn.Conv2D.html) tem suporte para dilatação dos filtros de acordo com o parâmetro *dilation*.


In [0]:
class LeNet(nn.Block):
    def __init__(self, classes=10, **kwargs):
        super(LeNet, self).__init__(**kwargs)
        with self.name_scope():
            self.conv1 = nn.Conv2D(6, kernel_size=5, strides=1, padding=0, activation='tanh')    # entrada: (b, 1, 32, 32) e saida: (b, 6, 28, 28)
            self.avgpool1 = nn.AvgPool2D(pool_size=2, strides=2)                                 # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)
            self.conv2 = nn.Conv2D(16, kernel_size=5, strides=1, padding=0, dilation=2, activation='tanh')   # entrada: (b, 6, 14, 14) e saida: (b, 16, 10, 10)
            # self.avgpool2 = nn.AvgPool2D(pool_size=2, strides=2)                                 # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)
            # self.conv3 = nn.Conv2D(120, kernel_size=5, strides=1, padding=0, activation='tanh')  # entrada: (b, 16, 5, 5) e saida: (b, 120, 1, 1)
            self.flatten = nn.Flatten()  # lineariza formando um vetor                           # entrada: (b, 120, 14, 14) e saida: (b, 120*20*20) = (b, 48000)
            self.fc1 = nn.Dense(84, activation="tanh")                                           # entrada: (b, 48000) e saida: (b, 84)
            self.fc2 = nn.Dense(classes)                                                         # entrada: (b, 84) e saida: (b, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        # print(x.shape)
        # x = self.avgpool2(x)
        # x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [0]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 10, 0.1, 128, 0.000001

# rede baseada na LeNet-5 
net = LeNet()
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd_lambda, 'momentum': 0.9})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 0.9418, train acc 0.635, test loss 0.4894, test acc 0.819, time 9.7 sec
epoch 2, train loss 0.4286, train acc 0.842, test loss 0.4128, test acc 0.851, time 9.4 sec
epoch 3, train loss 0.3843, train acc 0.861, test loss 0.3758, test acc 0.864, time 9.0 sec
epoch 4, train loss 0.3550, train acc 0.869, test loss 0.3709, test acc 0.862, time 9.1 sec
epoch 5, train loss 0.3444, train acc 0.873, test loss 0.3685, test acc 0.869, time 9.6 sec
epoch 6, train loss 0.3218, train acc 0.881, test loss 0.3679, test acc 0.870, time 9.0 sec
epoch 7, train loss 0.3169, train acc 0.883, test loss 0.3479, test acc 0.869, time 9.3 sec
epoch 8, train loss 0.2986, train acc 0.891, test loss 0.3165, test acc 0.887, time 9.4 sec
epoch 9, train loss 0.2872, train acc 0.893, test loss 0.3313, test acc 0.884, time 9.4 sec
epoch 10, train loss 0.2804, train acc 0.895, test loss 0.3290, test acc 0.882, time 9.4 sec


Para efeito de **comparação**, recriamos a rede sem as duas camadas convolucionais (removidas na arquitetura anterior) e sem camadas dilatadas. Dessa forma, podemos observar o ganho das convoluções dilatadas.

In [0]:
class LeNet(nn.Block):
    def __init__(self, classes=10, **kwargs):
        super(LeNet, self).__init__(**kwargs)
        with self.name_scope():
            self.conv1 = nn.Conv2D(6, kernel_size=5, strides=1, padding=0, activation='tanh')    # entrada: (b, 1, 32, 32) e saida: (b, 6, 28, 28)
            self.avgpool1 = nn.AvgPool2D(pool_size=2, strides=2)                                 # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)
            self.conv2 = nn.Conv2D(16, kernel_size=5, strides=1, padding=0, activation='tanh')   # entrada: (b, 6, 14, 14) e saida: (b, 16, 10, 10)
            # self.avgpool2 = nn.AvgPool2D(pool_size=2, strides=2)                                 # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)
            # self.conv3 = nn.Conv2D(120, kernel_size=5, strides=1, padding=0, activation='tanh')  # entrada: (b, 16, 5, 5) e saida: (b, 120, 1, 1)
            self.flatten = nn.Flatten()  # lineariza formando um vetor                           # entrada: (b, 120, 14, 14) e saida: (b, 120*20*20) = (b, 48000)
            self.fc1 = nn.Dense(84, activation="tanh")                                           # entrada: (b, 48000) e saida: (b, 84)
            self.fc2 = nn.Dense(classes)                                                         # entrada: (b, 84) e saida: (b, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.avgpool1(x)
        x = self.conv2(x)
        # print(x.shape)
        # x = self.avgpool2(x)
        # x = self.conv3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [4]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 10, 0.1, 128, 0.000001

# rede baseada na LeNet-5 
net = LeNet()
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr, 'wd': wd_lambda, 'momentum': 0.9})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 0.8227, train acc 0.687, test loss 0.4786, test acc 0.827, time 18.4 sec
epoch 2, train loss 0.4624, train acc 0.831, test loss 0.4566, test acc 0.836, time 18.2 sec
epoch 3, train loss 0.4282, train acc 0.846, test loss 0.4249, test acc 0.846, time 19.2 sec
epoch 4, train loss 0.4117, train acc 0.850, test loss 0.3881, test acc 0.860, time 12.7 sec
epoch 5, train loss 0.3879, train acc 0.862, test loss 0.3999, test acc 0.854, time 12.0 sec
epoch 6, train loss 0.3699, train acc 0.866, test loss 0.3547, test acc 0.869, time 11.8 sec
epoch 7, train loss 0.3642, train acc 0.868, test loss 0.3504, test acc 0.876, time 17.8 sec
epoch 8, train loss 0.3507, train acc 0.872, test loss 0.3876, test acc 0.866, time 17.7 sec
epoch 9, train loss 0.3413, train acc 0.875, test loss 0.3430, test acc 0.873, time 25.5 sec
epoch 10, train loss 0.3317, train acc 0.880, test loss 0.3769, test acc 0.861, time 29.6 sec


## Convolução Separável por Canal (*Depthwise Separable Convolution*)

Após o resurgimento das redes neurais, a cada nova arquitetura proposta, mais e mais camadas e parâmetros foram sendo usados. Por exemplo, a primeira arquitetura dessa nova onda, a [AlexNet](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=2ahUKEwi6usf1sqXjAhWFGLkGHXhPBC4QFjAAegQIAhAC&url=https%3A%2F%2Fpapers.nips.cc%2Fpaper%2F4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf&usg=AOvVaw2hqjjvSjIpuuCLLdojAmS5), tem **61.100.840** de parâmetros.

Claramente, pelo tamanho de parâmetros, arquitetura como essas não são passíveis de serem treinadas em dispositivos com menos memória, como sistema embarcados, celulares, tablets, etc.
Por isso, uma linha de pesquisa procura otimizar o uso dos parâmetros em rede neurais.
Para esse fim, essa linha de pesquisa propôs uma nova camada, chamada Convolução Separável por Canal (*Depthwise Separable Convolution*), que é capaz de reproduzir arquiteturas convolucionais, obtendo resultados similar, porém com menos parâmetros.

A ideia por trás dessa camada é representada abaixo. Ao invés de ter uma única camada de convolução que processará toda a entrada (ou seja, terá parâmetros para todos os canais de entrada), essa convolução é dividida em duas partes. Na primeira parte, cada canal é processado separadamente por um filtro específico. Formalmente, suponha que a entrada tenha $c_i$ canais.
Nesse caso, a primeira parte de convolução terá  $c_i$ filtros de tamanho $k\times k$, onde cada um desses filtros processará somente um canal da entrada, gerando $c_i$ *feature maps* de saída. Na segunda parte, esses *feature maps* serão processados por uma convolução $1\times 1$, que gerará a saída final esperada. Suponha que queiramos que essa camada gere $c_o$ canais de saída. Essa segunda parte terá $c_o$ filtros de $1\times 1$ que processarão os *feature maps* gerados na primeira parte, gerando a saída final com $c_o$ canais.

<p align="center">
  <img src="https://cdn-images-1.medium.com/max/800/1*Voah8cvrs7gnTDf6acRvDw.png">
</p>

Vamos examinar a diferença entre a convolução padrão e a convolução separável por canal em termos de número de parâmetros. Suponha que:

- n, seja a dimensão espacial (largura e altura), da entrada,
- k, seja largura e altura do filtro,
- c_i, seja o número de canais de entrada, e
- c_o, o número de canais de saída.

Uma convolução regular tem k * k * c_i * c_o parâmetros, porque, para cada canal de saída, há um filtro $k\times k$ para cada canal de entrada.
Em contrapartida, as convoluções separáveis por canal tem k * k * c_i parâmetros na primeira parte e, em seguida,  1 * 1 * c_i * c_o parâmetros para a segunda parte. Deve ser óbvio que, para um c_o não trivial, a soma desses dois é significativamente menor que k * k * c_i * c_o.

Agora, vamos comparar o custo computacional. Para uma convolução regular, realizamos K * K * c_i operações em cada posição da entrada (para calcular a convolução 2D em 3 dimensões). Para toda a entrada, o número de cálculos é, portanto, k * k * c_i * n * n e tomando todos os canais de saída, obtemos k * k * c_i * n * n * c_o.
Para convoluções separáveis em profundidade, precisamos de operações k * k * c_i * n * n para a parte de profundidade; então precisamos de n * n * c_i * c_o operações para a parte de mistura. 

Vamos usar alguns números reais para sentir a diferença.
Assumiremos 

- n = 128, 
- k = 3, 
- c_i = 3, 
- c_o = 16. 

Para convolução regular:

     Parâmetros: 3 * 3 * 3 * 16 = 432
     Custo de computação: 3 * 3 * 3 * 128 * 128 * 16 = ~ 7e6

Para a convolução separável em profundidade:

     Parâmetros: 3 * 3 * 3 + 3 * 16 = 75
     Custo de computação: 3 * 3 * 3 * 128 * 128 + 128 * 128 * 3 * 16 = ~ 1.2e6

Frameworks modernos implementam esse tipo de camada de forma diferente. No MXNet, nosso caso de estudo, a primeira parte dessa convolução é feita quando se coloca o [número de grupos da camada convolucional igual ao número de canais de entrada](https://github.com/apache/incubator-mxnet/issues/10142#issuecomment-373895855). Esse [parâmetro](https://beta.mxnet.io/api/gluon/_autogen/mxnet.gluon.nn.Conv2D.html) é usado para definir o agrupamento entre canais de entrada e filtros/neurônios. Logo, quando se tem um número de canais de entrada igual ao número de filtros, e também igual ao número de grupos. Dessa forma, cada filtro processará somente um canal.

Já a segunda parte dessa convolução é feita naturalmente, usando o *kernel*=1.

Abaixo, implementamos a LeNet-5 usando essa convolução.

<p align="center">
  <img width=700 src="https://engmrk.com/wp-content/uploads/2018/09/LeNEt_Summary_Table.jpg">
</p>


In [0]:
class LeNet(nn.Block):
    def __init__(self, classes=10, **kwargs):
        super(LeNet, self).__init__(**kwargs)
        with self.name_scope():
            self.conv1_1 = nn.Conv2D(1, kernel_size=5, strides=1, padding=0, groups=1)  # entrada: (b, 1, 32, 32) e saida: (b, 1, 28, 28)
            self.conv1_2 = nn.Conv2D(6, kernel_size=1, strides=1, padding=0, activation='tanh')            # entrada: (b, 1, 28, 28) e saida: (b, 6, 28, 28)
            self.avgpool1 = nn.AvgPool2D(pool_size=2, strides=2)                                           # entrada: (b, 6, 28, 28) e saida: (b, 6, 14, 14)
            
            self.conv2_1 = nn.Conv2D(6, kernel_size=5, strides=1, padding=0, groups=6)  # entrada: (b, 6, 14, 14) e saida: (b, 6, 10, 10)
            self.conv2_2 = nn.Conv2D(16, kernel_size=1, strides=1, padding=0, activation='tanh')           # entrada: (b, 6, 10, 10) e saida: (b, 16, 10, 10)
            self.avgpool2 = nn.AvgPool2D(pool_size=2, strides=2)                                           # entrada: (b, 16, 10, 10) e saida: (b, 16, 5, 5)
            
            self.conv3_1 = nn.Conv2D(16, kernel_size=5, strides=1, padding=0, groups=16) # entrada: (b, 16, 5, 5) e saida: (b, 16, 1, 1)
            self.conv3_2 = nn.Conv2D(120, kernel_size=1, strides=1, padding=0, activation='tanh')           # entrada: (b, 16, 1, 1) e saida: (b, 120, 1, 1)

            self.flatten = nn.Flatten()  # lineariza formando um vetor                           # entrada: (b, 120, 1, 1) e saida: (b, 120*1*1) = (b, 120)
            self.fc1 = nn.Dense(84, activation="tanh")                                           # entrada: (b, 120) e saida: (b, 84)
            self.fc2 = nn.Dense(classes)                                                         # entrada: (b, 84) e saida: (b, 10)

    def forward(self, x):
        # print('x', x.shape)
        x = self.conv1_1(x)
        # print('conv1_1', x.shape)
        x = self.conv1_2(x)
        # print('conv1_2', x.shape)
        x = self.avgpool1(x)
        x = self.conv2_1(x)
        # print('conv2_1', x.shape)
        x = self.conv2_2(x)
        # print('conv2_2', x.shape)
        x = self.avgpool2(x)
        x = self.conv3_1(x)
        # print('conv3_1', x.shape)
        x = self.conv3_2(x)
        # print('conv3_2', x.shape)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

## Se antes, essa arquitetura tinha 61.470 parâmetros, agora ela tem somente 13.517 (4,5 vezes menos)‬.

In [0]:
# parâmetros: número de epochs, learning rate (ou taxa de aprendizado), 
# tamanho do batch, e lambda do weight decay
num_epochs, lr, batch_size, wd_lambda = 10, 0.01, 128, 0.000001

# rede baseada na LeNet-5 
net = LeNet()
net.initialize(init.Normal(sigma=0.01), ctx=ctx)

# função de custo (ou loss)
loss = gloss.SoftmaxCrossEntropyLoss()

# carregamento do dado: mnist
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=32)

# trainer do gluon
trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': lr, 'wd': wd_lambda})

# treinamento e validação via MXNet
train_validate(net, train_iter, test_iter, batch_size, trainer, loss, 
               ctx, num_epochs)

training on gpu(0)
epoch 1, train loss 1.0537, train acc 0.587, test loss 0.6860, test acc 0.740, time 10.5 sec
epoch 2, train loss 0.6245, train acc 0.765, test loss 0.5870, test acc 0.789, time 10.1 sec
epoch 3, train loss 0.5336, train acc 0.803, test loss 0.5074, test acc 0.815, time 10.7 sec
epoch 4, train loss 0.4946, train acc 0.819, test loss 0.4978, test acc 0.812, time 10.5 sec
epoch 5, train loss 0.4833, train acc 0.823, test loss 0.4777, test acc 0.825, time 10.6 sec
epoch 6, train loss 0.4680, train acc 0.830, test loss 0.4781, test acc 0.820, time 10.4 sec
epoch 7, train loss 0.4522, train acc 0.836, test loss 0.4880, test acc 0.833, time 10.5 sec
epoch 8, train loss 0.4490, train acc 0.837, test loss 0.4469, test acc 0.837, time 10.5 sec
epoch 9, train loss 0.4443, train acc 0.839, test loss 0.4544, test acc 0.833, time 10.5 sec
epoch 10, train loss 0.4319, train acc 0.843, test loss 0.4293, test acc 0.845, time 10.6 sec
